# TODO
- Função de extração
    - "Escalonar", no momento ainda só esta pegando os resultados da primeira pagina de respostas.
    - Receber e lidar com um range de datas ao inves de uma data só (pelo que entendi do pdf isso é um requisito ne? Um range de data ao inves de uma data só)
    - Checar se o arquivo ja foi baixado. Basicamente criar uma condicional para evitar de ficar recriando csv files que possuem os mesmos dados.
- Função do DB
    - Estabelecer uma tabela de "status" contendo os nomes dos arquivos que ja foram jogados pra dentro do banco de dados.
    - Criar uma condicional para evitar lançamento de dados ja adicionados previamente no banco de dados.
- Análise
    - Basicamente um Jupyternotebook chamando as funções criadas nos scripts; com as análises basicas de dados  e as visualizações dessas análises
- Documentação
    - "Dar um tapa" na documentação.

In [1]:
# Importing Libraries
import sqlite3
import pandas as pd
import plotly.express as px
import source_to_landing
import landing_to_sql

In [2]:
# Query and Filter Functions
def query_db(sql_query:str) -> pd.DataFrame():
    """
    This function allows an easy implementation with SQL query into the historical flights database. Returning DataFrame with the Query's result.
    
    Args:
        sql_query(str): SQL Query.
        
    Returns:
        df(pd.DataFrame): pandas.DataFrame containing the query results.
    """
    connection = sqlite3.connect("database/historical_flights.db")
    df = pd.read_sql_query(sql_query, connection)
    return df


def filter_db(keyword:str ,option:str = "airline") -> pd.DataFrame():
    """
    This function filter the db according to the option selected and returns a DataFrame with the results.
    
    Args:
        keyword(str): Keyword used in filter.
        option(str): Filter option selected. Default value:"airline".
            Examples: "airline", "origin", "destination", "aircraft"
        
    Returns:
        df(pd.DataFrame): pandas.DataFrame containing the query results.
    """
    connection = sqlite3.connect("database/historical_flights.db")
    
    if option == "airline":
        query = f"SELECT * FROM flights WHERE airline_name = '{keyword}'"
        
    elif option == "origin":
        query = f"SELECT * FROM flights WHERE depart_airport = '{keyword}'"
        
    elif option == "destination":
        query = f"SELECT * FROM flights WHERE arrival_airport = '{keyword}'"
        
    elif option == "aircraft":
        query = f"SELECT * FROM flights WHERE aircraft_registration = '{keyword}'"
    
    df = pd.read_sql_query(query, connection)

    return df

## Receiveing and storing data

Brief description about the scripts created to retrieves data from the API and store into SQL database.

### `source_to_landing`

### `landing_to_sql`

In [4]:
#TODO
# Add more description about the scripts and let it with some example
csv_paths = source_to_landing.retrieve_hist_flights("2024-04-20","2024-04-22","dl","iata","YourAPI")

Taking data from offset:0
Taking data from offset:100
Taking data from offset:200
Taking data from offset:300
Taking data from offset:400
Taking data from offset:500
Taking data from offset:600
Taking data from offset:700
Taking data from offset:800
Taking data from offset:900
Taking data from offset:1000
Taking data from offset:1100
Taking data from offset:1200
Taking data from offset:1300
Taking data from offset:1400
Taking data from offset:1500
Taking data from offset:1600
Taking data from offset:1700
Taking data from offset:1800
Taking data from offset:1900
Taking data from offset:2000
Taking data from offset:2100
Taking data from offset:2200
Taking data from offset:2300
Taking data from offset:2400
Taking data from offset:2500
Taking data from offset:2600
Taking data from offset:2700
Taking data from offset:2800
Taking data from offset:2900
Taking data from offset:3000
Taking data from offset:3100
Taking data from offset:3200
Taking data from offset:3300
Taking data from offset:34

KeyError: 'data'

In [ ]:
for paths in csv_paths:
    landing_to_sql.upload_csv(path)

## Querying and filtering stored data

Brief description about the scripts created to query and fitler data from SQL db.

### `query_db`

### `filter_db`

In [ ]:
#TODO
# Add more descriptionabout the scripts and let it with some examples
query_db()
filter_db()

# Data Analysis

## Basic Statistics

In [ ]:
total_flights = query_db("SELECT COUNT(*) FROM flights")

In [ ]:
avg_flight_dur = query_db("SELECT ")

In [ ]:
flights_p_day = query_db("SELECT COUNT(*) FROM flights GROUP BY ")